In [1]:
import pandas as pd
import geopandas as gpd
import pyodbc
import sqlalchemy as sal
from shapely import wkb

In [2]:
shapefile =r'T:\DCProjects\Modeling\Sidewalk\data\bikeped.shp'
# read the entire shapefile first 
gdf = gpd.read_file(shapefile)

In [3]:
gdf.head(5)

,id,path_type,city,source,data,geometry
0,0,sidewalk,Eugene,EUG,sidewalks,"LINESTRING (4226331.689 893353.069, 4226329.75..."
1,1,sidewalk,Eugene,EUG,sidewalks,"LINESTRING (4226455.939 893192.738, 4226508.55..."
2,2,sidewalk,Eugene,EUG,sidewalks,"LINESTRING (4226404.210 893194.363, 4226455.93..."
3,3,sidewalk,Eugene,EUG,sidewalks,"LINESTRING (4226352.186 893197.835, 4226376.45..."
4,4,sidewalk,Eugene,EUG,sidewalks,"LINESTRING (4226252.842 893218.949, 4226276.62..."


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [5]:
constr = 'mssql+pyodbc://rliddb.int.lcog.org,5433/GIS_CLMPO?trusted_connection=yes&driver=SQL+Server+Native+Client+11.0'
engine = sal.create_engine(constr)
conn = engine.connect()

In [7]:
# define a function to convert to wkb 
def wkb_hexer(line):
    return line.wkb_hex

In [11]:
gdf.shape

(77617, 6)

In [13]:
gdf = gdf[gdf.geometry.notnull()]

In [14]:
gdf.shape

(77603, 6)

In [15]:
# apply to each row in the 'geometry' column
gdf['geometry'] = gdf['geometry'].apply(wkb_hexer)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [16]:
# finally, insert the dataframe into a table called 'bikeped' 
gdf.to_sql('bikeped', con = engine, schema = 'dbo', if_exists = 'replace')